In [ ]:
%env GOOGLE_CLOUD_PROJECT = physionet-data-403309 #Change to your own project

env: GOOGLE_CLOUD_PROJECT=physionet-data-403309 #Change to your own project


In [ ]:
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
from google.cloud import bigquery

import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%reload_ext google.colab.data_table
%load_ext google.colab.data_table

from google.colab import drive
drive.mount('/content/drive')

Authenticated
The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
Mounted at /content/drive


#Sites

In [ ]:
all_sites = {'Axillary', 'Blood', 'Esophogeal', 'Oral', 'Rectal', 'Temporal', 'Tympanic'}
sites_core ={'Axillary', 'Blood', 'Esophogeal','Oral', 'Rectal'}
sites_peripheral = {'Temporal', 'Tympanic'}
sites_core_filt ={'Esophogeal','Oral', 'Rectal'}
sites_peripheral_filt = {'Temporal', 'Tympanic'}

#Function to Upload Queries

In [ ]:
def create_table_id(project_id, dataset_id, tab_id):
  return f"{project_id}.{dataset_id}.{tab_id}"

#Function to access Google Sheets

In [ ]:
def get_vars_df(sheet_id, gid):

  return pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}')

#MIMIC-IV

##Create query with all temperature pairs
Only consider pairs of reference vs. IR sites

In [ ]:
def create_full_temperature_query(temperature_site_1, temperature_site_2):
    return f"""
WITH temperature_site_1_Readings AS (
  SELECT * FROM(
    SELECT
      ce.subject_id,
      ce.hadm_id AS hospital_admission_id,
      ce.stay_id,
      ce.charttime AS temperature_site_1_charttime,
      MAX(CASE WHEN ce.itemid = 224642 THEN value END) AS temperature_site_1_temperature_site,
      ROUND(CAST(
        AVG(CASE
            WHEN ce.itemid IN (223761)
                THEN (ce.valuenum - 32) / 1.8
            WHEN ce.itemid IN (223762)
                THEN ce.valuenum END)
        AS NUMERIC), 2) AS temperature_site_1_temperature
    FROM `physionet-data.mimiciv_icu.chartevents` ce
    WHERE ce.itemid IN (224642, 223762, 223761)
    GROUP BY ce.subject_id, ce.hadm_id, ce.stay_id, ce.charttime
  )
  WHERE temperature_site_1_temperature_site = "{temperature_site_1}"
)
,
temperature_site_2_Readings AS (
  SELECT * FROM (
    SELECT
      ce.subject_id,
      ce.hadm_id AS hospital_admission_id,
      ce.stay_id,
      ce.charttime AS temperature_site_2_charttime,
      MAX(CASE WHEN ce.itemid = 224642 THEN value END) AS temperature_site_2_temperature_site,
      ROUND(CAST(
        AVG(CASE
            WHEN ce.itemid IN (223761)
                THEN (ce.valuenum - 32) / 1.8
            WHEN ce.itemid IN (223762)
                THEN ce.valuenum END)
        AS NUMERIC), 2) AS temperature_site_2_temperature
    FROM `physionet-data.mimiciv_icu.chartevents` ce
    WHERE ce.itemid IN (224642, 223762, 223761)
    GROUP BY ce.subject_id, ce.hadm_id, ce.stay_id, ce.charttime
  )
  WHERE temperature_site_2_temperature_site = "{temperature_site_2}"
)

SELECT
  r.subject_id,
  r.hospital_admission_id,
  r.stay_id,
  r.temperature_site_1_charttime,
  r.temperature_site_1_temperature,
  r.temperature_site_1_temperature_site,
  t.temperature_site_2_charttime,
  t.temperature_site_2_temperature,
  t.temperature_site_2_temperature_site,
  (r.temperature_site_1_temperature - t.temperature_site_2_temperature) AS temperature_diff,
  ABS(TIMESTAMP_DIFF(r.temperature_site_1_charttime, t.temperature_site_2_charttime, HOUR)) AS time_diff_hours

FROM temperature_site_1_Readings r

INNER JOIN temperature_site_2_Readings t
ON r.stay_id = t.stay_id
WHERE r.temperature_site_1_temperature IS NOT NULL
  AND t.temperature_site_2_temperature IS NOT NULL
"""

In [ ]:
pairs_unfiltered = []
first_pair = 1

for s1 in sites_core:
    for s2 in sites_peripheral:
        if s1 != s2:
                query = create_full_temperature_query(s1, s2)
                client = bigquery.Client(project='physionet-data-403309')
                query_s1_s2 = client.query(query).to_dataframe()
                if first_pair:
                    pairs_unfiltered = query_s1_s2
                    first_pair = 0
                else:
                    pairs_unfiltered = pd.concat([pairs_unfiltered, query_s1_s2],
                                axis=0)

In [ ]:
len(pairs_unfiltered)

In [ ]:
len(pairs_unfiltered.groupby(by=["subject_id"]).first())

Upload query with unfiltered temperature pairs

In [ ]:
project_id = "physionet-data-403309"
dataset_id = "pairs"

final_tables = {"mimic_iv_original":  pairs_unfiltered}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

##Query filtered by temperature values

In [ ]:
%%bigquery pairs_filt_temperature

SELECT *
FROM `physionet-data-403309.pairs.mimic_iv_original`
WHERE temperature_site_1_temperature > 30
AND temperature_site_1_temperature < 45
AND temperature_site_2_temperature > 30
AND temperature_site_2_temperature < 45

Upload query

In [ ]:
project_id = "physionet-data-403309"
dataset_id = "pairs"

final_tables = {"mimic_iv_temp": pairs_filt_temperature}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

##Filtered by temperature values and time window (window <= 4h)

In [ ]:
%%bigquery pairs_filt_temperature_window

SELECT *
FROM `physionet-data-403309.pairs.mimic_iv_temp`
WHERE ABS(TIMESTAMP_DIFF(t.temperature_site_2_charttime, r.temperature_site_1_charttime, HOUR)) <= 4

Upload query

In [ ]:
project_id = "physionet-data-403309"
dataset_id = "pairs"

final_tables = {"mimic_iv_temp_w": pairs_filt_temperature_window}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

##Create query with filtered temperature pairs
Filter by:
- temperatures within ]30, 45[ ºC
- time window between measurements <= 4 hours
- sites_core = {'Oral', 'Esophogeal', 'Rectal'}
- sites_peripheral = {'Temporal', 'Tympanic'}

In [ ]:
def create_filtered_temperature_query(temperature_site_1, temperature_site_2, window):
    return f"""
WITH temperature_site_1_Readings AS (
  SELECT * FROM(
    SELECT
      ce.subject_id,
      ce.hadm_id AS hospital_admission_id,
      ce.stay_id,
      ce.charttime AS temperature_site_1_charttime,
      MAX(CASE WHEN ce.itemid = 224642 THEN value END) AS temperature_site_1_temperature_site,
      ROUND(CAST(
        AVG(CASE
            WHEN ce.itemid IN (223761)
                AND ce.valuenum > 86
                AND ce.valuenum < 113
                THEN (ce.valuenum - 32) / 1.8
            WHEN ce.itemid IN (223762)
                AND ce.valuenum > 30
                AND ce.valuenum < 45
                THEN ce.valuenum END)
        AS NUMERIC), 2) AS temperature_site_1_temperature
    FROM `physionet-data.mimiciv_icu.chartevents` ce
    WHERE ce.itemid IN (224642, 223762, 223761)
    GROUP BY ce.subject_id, ce.hadm_id, ce.stay_id, ce.charttime
  )
  WHERE temperature_site_1_temperature_site = "{temperature_site_1}"
)
,
temperature_site_2_Readings AS (
  SELECT * FROM (
    SELECT
      ce.subject_id,
      ce.hadm_id AS hospital_admission_id,
      ce.stay_id,
      ce.charttime AS temperature_site_2_charttime,
      MAX(CASE WHEN ce.itemid = 224642 THEN value END) AS temperature_site_2_temperature_site,
      ROUND(CAST(
        AVG(CASE
            WHEN ce.itemid IN (223761)
                AND ce.valuenum > 86
                AND ce.valuenum < 113
                THEN (ce.valuenum - 32) / 1.8
            WHEN ce.itemid IN (223762)
                AND ce.valuenum > 30
                AND ce.valuenum < 45
                THEN ce.valuenum END)
        AS NUMERIC), 2) AS temperature_site_2_temperature
    FROM `physionet-data.mimiciv_icu.chartevents` ce
    WHERE ce.itemid IN (224642, 223762, 223761)
    GROUP BY ce.subject_id, ce.hadm_id, ce.stay_id, ce.charttime
  )
  WHERE temperature_site_2_temperature_site = "{temperature_site_2}"
)

SELECT
  r.subject_id,
  r.hospital_admission_id,
  r.stay_id,
  r.temperature_site_1_charttime,
  r.temperature_site_1_temperature,
  r.temperature_site_1_temperature_site,
  t.temperature_site_2_charttime,
  t.temperature_site_2_temperature,
  t.temperature_site_2_temperature_site,
  (r.temperature_site_1_temperature - t.temperature_site_2_temperature) AS temperature_diff,
  ABS(TIMESTAMP_DIFF(r.temperature_site_1_charttime, t.temperature_site_2_charttime, HOUR)) AS time_diff_hours

FROM temperature_site_1_Readings r

INNER JOIN temperature_site_2_Readings t
ON r.stay_id = t.stay_id
WHERE ABS(TIMESTAMP_DIFF(t.temperature_site_2_charttime, r.temperature_site_1_charttime, HOUR)) <= {window}
  AND r.temperature_site_1_temperature IS NOT NULL
  AND t.temperature_site_2_temperature IS NOT NULL
"""

In [ ]:
pairs_filtered = []
window_time = [4]
first_pair = 1

for s1 in sites_core_filt:
    for s2 in sites_peripheral_filt:
        for h in window_time:
            query = create_filtered_temperature_query(s1, s2, h)
            client = bigquery.Client(project='physionet-data-403309')
            query_s1_s2 = client.query(query).to_dataframe()
            if first_pair:
                pairs_filtered = query_s1_s2
                first_pair = 0
            else:
                pairs_filtered = pd.concat([pairs_filtered, query_s1_s2],
                            axis=0)

Other approach:

In [ ]:
%%bigquery pairs_filt_temperature_window_site

SELECT *
FROM `physionet-data-403309.pairs.mimic_iv_temp_w`
WHERE temperature_site_1_temperature_site IN ('Esophogeal','Oral', 'Rectal')
AND temperature_site_2_temperature_site IN ('Temporal', 'Tympanic')

Upload query with **filtered** temperature pairs

In [ ]:
project_id = "physionet-data-403309"
dataset_id = "pairs"

final_tables = {"mimic_iv_temp_w_site": pairs_filt_temperature_window_site}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

##Patients information

In [ ]:
%%bigquery mimic_iv_pat --project physionet-data-403309

SELECT DISTINCT
    icu.subject_id
  , icu.hadm_id AS hospital_admission_id
  , icu.stay_id
  , "mimic_iv" AS source_db
  , 9999 AS hospitalid
  , ">= 500" AS numbedscategory
  , TRUE AS teachingstatus
  , "Northeast" AS region
  , CASE WHEN icu.admission_age > 90 THEN 90 ELSE icu.admission_age END AS age_admission
  , CASE WHEN icu.gender = "F" THEN 1 ELSE 0 END AS sex_female
  , weight.weight_admit AS weight_admission
  , height.height AS height_admission
  , weight.weight_admit / (POWER(height.height/100, 2)) AS BMI_admission
  , icu.admittime AS datetime_hospital_admit
  , icu.dischtime AS datetime_hospital_discharge
  , icu.icu_intime AS datetime_icu_admit
  , icu.icu_outtime AS datetime_icu_discharge
  , icu.los_hospital
  , icu.los_icu AS los_ICU
  , "Charlson" AS comorbidity_score_name
  , charlson.charlson_comorbidity_index AS comorbidity_score_value
  , ad.hospital_expire_flag AS in_hospital_mortality
  , icu.race AS race_ethnicity

-- ICU stays
FROM physionet-data.mimiciv_derived.icustay_detail
AS icu

-- Age
LEFT JOIN physionet-data.mimiciv_hosp.patients
AS pat
ON icu.subject_id = pat.subject_id

-- Weight
LEFT JOIN physionet-data.mimiciv_derived.first_day_weight
AS weight
ON icu.stay_id = weight.stay_id

-- Height
LEFT JOIN physionet-data.mimiciv_derived.first_day_height
AS height
ON icu.stay_id = height.stay_id

-- Admissions
LEFT JOIN physionet-data.mimiciv_hosp.admissions
AS ad
ON icu.hadm_id = ad.hadm_id

-- Charlson
LEFT JOIN physionet-data.mimiciv_derived.charlson
AS charlson
ON icu.hadm_id = charlson.hadm_id

Map races/ethnicities

In [ ]:
sheets_id = "1Hv_sOd0--6TPYiB3Crjdn_JrIhIazXXJc05mL4GefOU"
gid = "1877090341"

mimic_iv_map = get_vars_df(sheets_id, gid)

mapping_series = mimic_iv_map.set_index('original')['mapping']

# Map the 'race_ethnicity' column in the 'mimic_iv_pat' DataFrame
mimic_iv_pat['race_ethnicity'] = mimic_iv_pat['race_ethnicity'].map(mapping_series)

Correct data types

In [ ]:
for col in ['hospital_admission_id','stay_id']:
    mimic_iv_pat[col]=mimic_iv_pat[col].astype(int)

Upload query with patients information

In [ ]:
project_id = "physionet-data-403309"
dataset_id = "patients"

final_tables = {"mimic_iv_pat": mimic_iv_pat}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

Upload query with pairs + patients information

In [ ]:
%%bigquery pairs_filt_temp_w_site --project physionet-data-403309

SELECT *
FROM `physionet-data-403309.pairs.mimic_iv_temp_w_site`

In [ ]:
mimic_iv_pairs_pat = pairs_filt_temp_w_site.merge(
                  mimic_iv_pat.drop(["subject_id"], axis=1),
                  on=['hospital_admission_id', 'stay_id']
                    )

In [ ]:
project_id = "physionet-data-403309"
dataset_id = "pairs_pat"

final_tables = {"mimic_iv_temp_w_site": mimic_iv_pairs_pat}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 3711.77it/s]


##Query with pairs + clinical data

In [ ]:
def create_subquery(pairs_table,
                    temp_time,
                    var_type,
                    var_name,
                    var_new_name,
                    var_window_low_hr,
                    var_window_upper_hr,
                    var_source_table,
                    var_foreign_key,
                    var_timestamp):

    if temp_time == "temperature_site_1_charttime":
      time_diff = f"TIMESTAMP_DIFF(CAST(table.{var_timestamp} AS TIMESTAMP), CAST(pairs.{temp_time} AS TIMESTAMP), MINUTE)"
    elif temp_time == "nursingchartoffset_1":
      time_diff = f"table.{var_timestamp} - CAST(pairs.{temp_time} AS INT64) "

    return f"""
    {var_type}_{var_name} AS (
      SELECT * FROM (
        SELECT
          pairs.{var_foreign_key},
          {var_name},
          pairs.{temp_time},
          {time_diff} AS delta_{var_type}_{var_new_name},
          ROW_NUMBER() OVER(PARTITION BY pairs.{var_foreign_key}, pairs.{temp_time}
                            ORDER BY ABS({time_diff}) ASC) AS seq
        FROM `{pairs_table}` pairs
        LEFT JOIN `{var_source_table}` AS table
        ON table.{var_foreign_key} = CAST(pairs.{var_foreign_key} AS INT)
        AND {var_name} IS NOT NULL
        AND {time_diff} BETWEEN {var_window_low_hr} * 60 AND {var_window_upper_hr} * 60
      )
      WHERE seq = 1
    )
    """

In [ ]:
def create_complete_query(pairs_table, vars_df, temp_time):

  subquery_list = []

  # Iterate over each row in df
  for index, row in vars_df.iterrows():

      subquery = create_subquery(pairs_table,
                                 temp_time,
                                 row['var_type'],
                                 row['var_name'],
                                 row['var_new_name'],
                                 row['var_window_low_hr'],
                                 row['var_window_upper_hr'],
                                 row['var_source_table'],
                                 row['var_foreign_key'],
                                 row['var_timestamp'])
      subquery_list.append(subquery)

  # Now you have your subqueries, join them into one string
  subquery_string = ",\n\n".join(subquery_list)

  final_query = f"""
  WITH
  {subquery_string}
  SELECT
      pairs.*,
      {" , ".join([f"{row.var_type}_{row.var_name}.delta_{row.var_type}_{row.var_new_name}, {row.var_type}_{row.var_name}.{row.var_name} AS {row.var_type}_{row.var_new_name}" for index, row in vars_df.iterrows()])}
  FROM `{pairs_table}` pairs
  {" ".join([f"LEFT JOIN {row.var_type}_{row.var_name} ON {row.var_type}_{row.var_name}.{row.var_foreign_key} = pairs.{row.var_foreign_key} AND {row.var_type}_{row.var_name}.{temp_time} = pairs.{temp_time}" for index, row in vars_df.iterrows()])}
  """
  return final_query

In [ ]:
temp_time = "temperature_site_1_charttime"
pairs_table = "physionet-data-403309.pairs_pat.mimic_iv_temp_w_site"

sheet_id = "1zgW22LTDMR60Al9da7R7UCRdfLGJ3HPoe3J-H-rz9Mw"
gid = "0"
vars_df = get_vars_df(sheet_id, gid)

query = create_complete_query(pairs_table, vars_df, temp_time)
client = bigquery.Client(project='physionet-data-403309')
mimic_iv_df = client.query(query).to_dataframe()

In [ ]:
with pd.option_context('display.max_columns', None):
    display(mimic_iv_df)

,subject_id,hospital_admission_id,stay_id,temperature_site_1_charttime,temperature_site_1_temperature,temperature_site_1_temperature_site,temperature_site_2_charttime,temperature_site_2_temperature,temperature_site_2_temperature_site,temperature_diff,time_diff_hours,source_db,hospitalid,numbedscategory,teachingstatus,region,age_admission,sex_female,weight_admission,height_admission,BMI_admission,datetime_hospital_admit,datetime_hospital_discharge,datetime_icu_admit,datetime_icu_discharge,los_hospital,los_ICU,comorbidity_score_name,comorbidity_score_value,in_hospital_mortality,race_ethnicity,delta_vitals_heart_rate,vitals_heart_rate,delta_vitals_resp_rate,vitals_resp_rate,delta_vitals_mbp_ni,vitals_mbp_ni,delta_vitals_sbp_ni,vitals_sbp_ni,delta_vitals_dbp_ni,vitals_dbp_ni,delta_vitals_mbp_i,vitals_mbp_i,delta_vitals_sbp_i,vitals_sbp_i,delta_vitals_dbp_i,vitals_dbp_i,delta_vitals_spo2,vitals_spo2,delta_cbc_hemoglobin,cbc_hemoglobin,delta_cbc_hematocrit,cbc_hematocrit,delta_cbc_mch,cbc_mch,delta_cbc_mchc,cbc_mchc,delta_cbc_mcv,cbc_mcv,delta_cbc_platelet,cbc_platelet,delta_cbc_rbc,cbc_rbc,delta_cbc_rdw,cbc_rdw,delta_cbc_wbc,cbc_wbc,delta_coag_fibrinogen,coag_fibrinogen,delta_coag_inr,coag_inr,delta_coag_pt,coag_pt,delta_coag_ptt,coag_ptt,delta_bmp_sodium,bmp_sodium,delta_bmp_potassium,bmp_potassium,delta_bmp_chloride,bmp_chloride,delta_bmp_bicarbonate,bmp_bicarbonate,delta_bmp_bun,bmp_bun,delta_bmp_creatinine,bmp_creatinine,delta_bmp_glucose,bmp_glucose,delta_bmp_aniongap,bmp_aniongap,delta_bmp_calcium,bmp_calcium,delta_bmp_lactate,bmp_lactate,delta_hfp_alt,hfp_alt,delta_hfp_alp,hfp_alp,delta_hfp_ast,hfp_ast,delta_hfp_bilirubin_total,hfp_bilirubin_total,delta_hfp_bilirubin_direct,hfp_bilirubin_direct,delta_hfp_albumin,hfp_albumin,delta_others_ck_cpk,others_ck_cpk,delta_others_ck_mb,others_ck_mb,delta_others_ld_ldh,others_ld_ldh,delta_sofa_past_overall_24hr,sofa_past_overall_24hr,delta_sofa_past_coagulation_24hr,sofa_past_coagulation_24hr,delta_sofa_past_liver_24hr,sofa_past_liver_24hr,delta_sofa_past_cardiovascular_24hr,sofa_past_cardiovascular_24hr,delta_sofa_past_cns_24hr,sofa_past_cns_24hr,delta_sofa_past_renal_24hr,sofa_past_renal_24hr,delta_sofa_future_overall_24hr,sofa_future_overall_24hr,delta_sofa_future_coagulation_24hr,sofa_future_coagulation_24hr,delta_sofa_future_liver_24hr,sofa_future_liver_24hr,delta_sofa_future_cardiovascular_24hr,sofa_future_cardiovascular_24hr,delta_sofa_future_cns_24hr,sofa_future_cns_24hr,delta_sofa_future_renal_24hr,sofa_future_renal_24hr,delta_extra_arcticsun_present,extra_arcticsun_present,delta_extra_dialysis_present,extra_dialysis_present,delta_extra_transfusion_present,extra_transfusion_present,delta_extra_norep_eq_dose,extra_norep_eq_dose
0,13651886,28215550,38853942,2111-06-02 16:00:00,36.610000000,Oral,2111-06-02 20:00:00,36.610000000,Temporal,0E-9,4,mimic_iv,9999,>= 500,True,Northeast,90,0,63.0,None,nan,2111-06-02 01:49:00,2111-06-11 15:03:00,2111-06-02 03:16:00,2111-06-03 07:33:04,9,1.170000000,Charlson,15,0,White,0,84.0,0,26.0,-59,81.0,-59,103.0,-59,71.0,<NA>,NaN,<NA>,NaN,<NA>,NaN,0,95.0,-232,9.0,-232,29.7,-232,26.3,-232,30.3,-232,87.0,-232,221.0,-232,3.42,-232,17.6,-232,7.0,<NA>,NaN,-232,3.7,-232,40.1,-232,55.3,-232,139.0,-232,4.7,-232,98.0,-232,26.0,-232,81.0,-232,3.3,-232,125.0,-232,15.0,-232,7.9,<NA>,NaN,-232,53.0,-232,947.0,-232,110.0,-232,4.8,<NA>,NaN,-1015,2.1,<NA>,NaN,-684,1.0,-232,216.0,-60,6,-60,0,-60,2,-60,1,-60,0,-60,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None
1,14683399,22818662,35577020,2149-01-19 15:00:00,36.500000000,Oral,2149-01-19 12:00:00,36.110000000,Temporal,0.390000000,3,mimic_iv,9999,>= 500,True,Northeast,88,0,65.3,None,nan,2149-01-13 23:31:00,2149-02-05 17:42:00,2149-01-17 09:01:49,2149-01-20 15:37:40,23,3.250000000,Charlson,15,0,White,0,83.0,0,16.0,<NA>,NaN,<NA>,NaN,<NA>,NaN,0,96.0,0,158.0,0,64.0,0,93.0,-656,8.4,-656,26.0,-656,34.0,-656,32.3,-656,105.0,-656,73.0,-656,2.47,-656,21.9,-656,1.7,<NA>,NaN,-6

In [ ]:
project_id = "physionet-data-403309"
dataset_id = "h_merged"

final_tables_merged = {"mimic_iv_temp_w_site":  mimic_iv_df}

for t_name, t_df in final_tables_merged.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 2790.62it/s]


#MIMIC III

##Patients

In [ ]:
%%bigquery mimic_iii_pat --project=physionet-data-403309

SELECT DISTINCT
  icu.subject_id
, icu.hadm_id AS hospital_admission_id
, icu.icustay_id
, "mimic_iii" AS source_db
, 9999 AS hospitalid
, ">= 500" AS numbedscategory
, TRUE AS teachingstatus
, "Northeast" AS region
, CASE WHEN icu.admission_age > 90 THEN 90 ELSE icu.admission_age END AS admission_age
, CASE WHEN icu.gender = "F" THEN 1 ELSE 0 END AS sex_female

, heightweight.weight_first as weight_admission
, heightweight.height_first as height_admission
, heightweight.weight_first / (POWER(heightweight.height_first/100, 2)) AS BMI_admission

, icu.admittime AS datetime_hospital_admit
, icu.dischtime AS datetime_hospital_discharge
, icu.intime AS datetime_icu_admit
, icu.outtime AS datetime_icu_discharge

, icu.los_hospital
, icu.los_icu AS los_ICU

, "Elixhauser" AS comorbidity_score_name
, elixhauser_vanwalraven AS comorbidity_score_value
, icu.hospital_expire_flag AS in_hospital_mortality
, icu.ethnicity AS race_ethnicity


FROM `physionet-data.mimiciii_clinical.icustays`
AS stays

-- ICU stays
LEFT JOIN physionet-data.mimiciii_derived.icustay_detail
AS icu
ON stays.subject_id = icu.subject_id
AND stays.hadm_id = icu.hadm_id
AND stays.icustay_id = icu.icustay_id

-- Hospital Admissions
LEFT JOIN physionet-data.mimiciii_clinical.admissions
AS ad
ON ad.hadm_id = icu.hadm_id

-- Height and Weight
LEFT JOIN physionet-data.mimiciii_derived.heightweight
AS heightweight
ON heightweight.icustay_id = icu.icustay_id

-- Elixhauser
LEFT JOIN physionet-data.mimiciii_derived.elixhauser_quan_score
AS elix
ON elix.hadm_id = icu.hadm_id


Map races/ethnicities

In [ ]:
sheets_id = "1Hv_sOd0--6TPYiB3Crjdn_JrIhIazXXJc05mL4GefOU"
gid = "541230090"

mimic_iii_map = get_vars_df(sheets_id, gid)

mapping_series = mimic_iii_map.set_index('original')['mapping']

# Map the 'race_ethnicity' column in the 'mimic_iv_pat' DataFrame
mimic_iii_pat['race_ethnicity'] = mimic_iii_pat['race_ethnicity'].map(mapping_series)
mimic_iii_pat['race_ethnicity'] = mimic_iii_pat['race_ethnicity'].fillna('Unknown')

In [ ]:
for col in ['hospital_admission_id','icustay_id']:
    mimic_iii_pat = mimic_iii_pat[mimic_iii_pat[col].notnull()]

In [ ]:
len(mimic_iii_pat)

61051

Upload query with patients information

In [ ]:
project_id = "physionet-data-403309" #change project name
dataset_id = "patients"

final_tables = {"mimic_iii_pat": mimic_iii_pat}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 777.30it/s]


Correct data types

In [ ]:
for col in ['hospital_admission_id','icustay_id']:
    mimic_iii_pat[col]=mimic_iii_pat[col].astype(int)

##All temperature pairs
Only consider pairs of reference vs. IR sites

In [ ]:
def create_full_temperature_query(temperature_site_1, temperature_site_2):
    return f"""
WITH temperature_site_1_Readings AS (
  SELECT * FROM(
    SELECT
      ce.subject_id,
      ce.hadm_id AS hospital_admission_id,
      ce.icustay_id,
      ce.charttime AS temperature_site_1_charttime,
      MAX(CASE WHEN ce.itemid = 224642 THEN value END) AS temperature_site_1_temperature_site,
      ROUND(CAST(
        AVG(CASE
            WHEN ce.itemid IN (223761)
                THEN (ce.valuenum - 32) / 1.8
            WHEN ce.itemid IN (223762)
                THEN ce.valuenum END)
        AS NUMERIC), 2) AS temperature_site_1_temperature
    FROM `physionet-data.mimiciii_clinical.chartevents` ce
    WHERE ce.itemid IN (224642, 223762, 223761)
    GROUP BY ce.subject_id, ce.hadm_id, ce.icustay_id, ce.charttime
  )
  WHERE temperature_site_1_temperature_site = "{temperature_site_1}"
)
,
temperature_site_2_Readings AS (
  SELECT * FROM (
    SELECT
      ce.subject_id,
      ce.hadm_id AS hospital_admission_id,
      ce.icustay_id,
      ce.charttime AS temperature_site_2_charttime,
      MAX(CASE WHEN ce.itemid = 224642 THEN value END) AS temperature_site_2_temperature_site,
      ROUND(CAST(
        AVG(CASE
            WHEN ce.itemid IN (223761)
                THEN (ce.valuenum - 32) / 1.8
            WHEN ce.itemid IN (223762)
                THEN ce.valuenum END)
        AS NUMERIC), 2) AS temperature_site_2_temperature
    FROM `physionet-data.mimiciii_clinical.chartevents` ce
    WHERE ce.itemid IN (224642, 223762, 223761)
    GROUP BY ce.subject_id, ce.hadm_id, ce.icustay_id, ce.charttime
  )
  WHERE temperature_site_2_temperature_site = "{temperature_site_2}"
)

SELECT
  r.subject_id,
  r.hospital_admission_id,
  r.icustay_id,
  r.temperature_site_1_charttime,
  r.temperature_site_1_temperature,
  r.temperature_site_1_temperature_site,
  t.temperature_site_2_charttime,
  t.temperature_site_2_temperature,
  t.temperature_site_2_temperature_site,
  (r.temperature_site_1_temperature - t.temperature_site_2_temperature) AS temperature_diff,
  ABS(TIMESTAMP_DIFF(r.temperature_site_1_charttime, t.temperature_site_2_charttime, HOUR)) AS time_diff_hours

FROM temperature_site_1_Readings r

INNER JOIN temperature_site_2_Readings t
ON r.icustay_id = t.icustay_id
WHERE r.temperature_site_1_temperature IS NOT NULL
  AND t.temperature_site_2_temperature IS NOT NULL
"""

In [ ]:
pairs_unfiltered = []
first_pair = 1

for s1 in sites_core:
    for s2 in sites_peripheral:
        if s1 != s2:
                query = create_full_temperature_query(s1, s2)
                client = bigquery.Client(project='physionet-data-403309')
                query_s1_s2 = client.query(query).to_dataframe()
                if first_pair:
                    pairs_unfiltered = query_s1_s2
                    first_pair = 0
                else:
                    pairs_unfiltered = pd.concat([pairs_unfiltered, query_s1_s2],
                                axis=0)

In [ ]:
len(pairs_unfiltered) #number of pairs

6402

In [ ]:
len(pairs_unfiltered.groupby(by=["subject_id"]).first()) #number of subjects

57

Upload query with unfiltered temperature pairs

In [ ]:
project_id = "physionet-data-403309" #change project name
dataset_id = "pairs"

final_tables = {"mimic_iii_original":  pairs_unfiltered}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 717.34it/s]


##Query filtered by temperature values

In [ ]:
%%bigquery pairs_filt_temperature --project=physionet-data-403309

SELECT *
FROM `physionet-data-403309.pairs.mimic_iii_original`
WHERE CAST(temperature_site_1_temperature AS FLOAT64) > 30
  AND CAST(temperature_site_1_temperature AS FLOAT64) < 45
  AND CAST(temperature_site_2_temperature AS FLOAT64) > 30
  AND CAST(temperature_site_2_temperature AS FLOAT64) < 45

Upload query

In [ ]:
project_id = "physionet-data-403309" #change project name
dataset_id = "pairs"

final_tables = {"mimic_iii_temp": pairs_filt_temperature}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 1618.17it/s]


##Filtered by temperature values and time window (window <= 4h)

In [ ]:
%%bigquery pairs_filt_temperature_window --project=physionet-data-403309

SELECT *
FROM `physionet-data-403309.pairs.mimic_iii_temp` AS t
WHERE ABS(TIMESTAMP_DIFF(TIMESTAMP(t.temperature_site_2_charttime), TIMESTAMP(t.temperature_site_1_charttime), HOUR)) <= 4

In [ ]:
project_id = "physionet-data-403309" #change project name
dataset_id = "pairs"

final_tables = {"mimic_iii_temp_w": pairs_filt_temperature_window}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 1754.94it/s]


##Create query with filtered temperature pairs
Filter by:
- temperatures within ]30, 45[ ºC
- time window between measurements <= 4 hours
- sites_core = {'Oral', 'Esophogeal', 'Rectal'}
- sites_peripheral = {'Temporal', 'Tympanic'}

In [ ]:
%%bigquery pairs_filt_temperature_window_site --project=physionet-data-403309

SELECT *
FROM `physionet-data-403309.pairs.mimic_iii_temp_w`
WHERE temperature_site_1_temperature_site IN ('Esophogeal','Oral', 'Rectal')
AND temperature_site_2_temperature_site IN ('Temporal', 'Tympanic')

In [ ]:
len(pairs_filt_temperature_window_site)

114

Upload query with **filtered** temperature pairs

In [ ]:
project_id = "physionet-data-403309" #change project name
dataset_id = "pairs"

final_tables = {"mimic_iii_temp_w_site": pairs_filt_temperature_window_site}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 8371.86it/s]


##Pairs + patients

In [ ]:
for col in ['hospital_admission_id','icustay_id']:
    pairs_filt_temperature_window_site[col] = pairs_filt_temperature_window_site[col].astype(int)

In [ ]:
mimic_iii_pairs_pat = pairs_filt_temperature_window_site.merge(
                  mimic_iii_pat.drop(["subject_id"], axis=1),
                  on=['hospital_admission_id', 'icustay_id']
                    )

Upload query with **filtered** temperature pairs + patients information

In [ ]:
project_id = "physionet-data-403309" #change project name
dataset_id = "pairs_pat"

final_tables = {"mimic_iii_temp_w_site": mimic_iii_pairs_pat}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 7598.38it/s]


##Merge horizontally
The result is the final table with the filtered temperature pairs + (extra) variables + patient information

In [ ]:
temp_time = "temperature_site_1_charttime"
pairs_table = "physionet-data-403309.pairs_pat.mimic_iii_temp_w_site"

sheet_id = "1zgW22LTDMR60Al9da7R7UCRdfLGJ3HPoe3J-H-rz9Mw"
gid = "1236068922"
vars_df = get_vars_df(sheet_id, gid)

query = create_complete_query(pairs_table, vars_df, temp_time)
client = bigquery.Client(project='physionet-data-403309')
mimic_iii_df = client.query(query).to_dataframe()

Upload merged table

In [ ]:
project_id = "physionet-data-403309" #change project name
dataset_id = "h_merged"

final_tables_merged = {"mimic_iii_temp_w_site":  mimic_iii_df}

for t_name, t_df in final_tables_merged.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 7639.90it/s]


#eICU

##Patients

In [ ]:
%%bigquery eicu_pat --project physionet-data-403309

SELECT DISTINCT
  icu.uniquepid AS subject_id
, icu.patienthealthsystemstayid AS hospital_admission_id
, icu.patientunitstayid AS stay_id
, "eicu" AS source_db
, icu.hospitalid
, hosp.numbedscategory
, hosp.teachingstatus
, icu.region
, CASE
    WHEN icu.age = '> 89' THEN CAST(90 AS STRING)
    ELSE icu.age
  END AS age_admission
, CASE WHEN pat.gender = "Female" THEN 1 ELSE 0 END AS sex_female

, icu.admissionweight AS weight_admission
, icu.admissionheight AS height_admission
, CASE
    WHEN icu.admissionheight > 0
    THEN icu.admissionweight / (POWER(icu.admissionheight/100, 2))
    ELSE NULL
  END AS BMI_admission

, icu.hospitaladmitoffset AS datetime_hospital_admit
, icu.hospitaldischargeoffset AS date_hospital_discharge
, icu.unitadmitoffset AS date_icu_admit
, icu.unitdischargeoffset AS date_icu_discharge

, (icu.hospitaldischargeoffset - icu.hospitaladmitoffset)/60/24 AS los_hospital
, icu.icu_los_hours / 24 AS los_ICU

, "Charlson" AS comorbidity_score_name
, cci.charlson_score AS comorbidity_score_value
, icu.hosp_mort AS in_hospital_mortality
, pat.ethnicity AS race_ethnicity


FROM `physionet-data.eicu_crd_derived.icustay_detail`
AS icu

LEFT JOIN `physionet-data.eicu_crd.patient`
AS pat
ON pat.patienthealthsystemstayid = icu.patienthealthsystemstayid

LEFT JOIN `physionet-data.eicu_crd.hospital`
AS hosp
ON pat.hospitalid = hosp.hospitalid

LEFT JOIN `physionet-data-403309.eicu_ExtraTables.cci`
AS cci
ON cci.patientunitstayid = icu.patientunitstayid

In [ ]:
len(eicu_pat)

200859

In [ ]:
sheets_id = "1Hv_sOd0--6TPYiB3Crjdn_JrIhIazXXJc05mL4GefOU"
gid = "622977158"

eicu_map = get_vars_df(sheets_id, gid)

mapping_series = eicu_map.set_index('original')['mapping']

# Map the 'race_ethnicity' column in the 'eicu_map' DataFrame
eicu_pat['race_ethnicity'] = eicu_pat['race_ethnicity'].map(mapping_series)
eicu_pat['race_ethnicity'] = eicu_pat['race_ethnicity'].fillna('Unknown')

In [ ]:
eicu_pat['race_ethnicity'].unique()

array(['White', 'Hispanic OR Latino', 'Unknown', 'Black', 'Asian',
       'American Indian / Alaska Native'], dtype=object)

In [ ]:
project_id = "physionet-data-403309"
dataset_id = "patients"

final_tables_merged_ethn = {"eicu_pat":  eicu_pat}

for t_name, t_df in final_tables_merged_ethn.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 951.31it/s]


##Temperature pairs

In [ ]:
dict_mappings = {}

In [ ]:
dict_mappings['eicu'] = {
    'Temporal':[
        'Temporal',
        'te,poral',
 'teemporal',
 'tem',
 'tem[oral',
 'tem[poral',
 'temoporal',
 'temoporol',
 'temopral',
 'temoral',
 'temp',
 'temp  art',
 'temp art',
 'temp artery',
 'temp at',
 'temp, art',
 'temp.',
 'tempa rt',
 'tempart',
 'temperal',
 'temple',
 'tempm art',
 'tempoal',
 'tempoarl',
 'tempoeral',
 'tempoiral',
 'tempooral',
 'tempor',
 'tempora',
 'tempora;',
 'temporak',
 'temporal',
 'temporal ',
 'temporal  artery',
 'temporal art',
 'temporal artery',
 'temporal artery scan',
 'temporal scan',
 'temporal,',
 'temporalf',
 'temporarl',
 'temporol',
 'temporral',
 'temporsl',
 'tempota',
 'tempral',
 'temprl',
 'temproal',
 'temproral',
 'tenporal',
 'termporal',
 'tewmporal',
 'tmp',
 'tmporal',
'.ta',
        'ta',
       'forehead',
        ],
    'Oral':[
        'Oral',
         'o',
 'o.',
 'oal',
 'oarl',
 'olral',
 'or',
 'or ',
 'or36.6al',
 'ora',
 'ora;',
 'orak',
 'oral',
 'oral ',
 'oral/axillary',
 'oral3.',
 'oral49',
 'orala',
 'orall',
 'orally',
 'orl',
 'orla',
 'orsl',
 'otal',
        '.oral'

        ],
    'Esophageal': [
        'esophageal','esophageal probe', 'esoph', 'eso', 'esophageal ', 'esophogeal'
    ],
    'Core':[
        'Core','core',
        'pa catheter',  'temperature sensing urinary catheter', 'intravascular (swan)','core urinary catheter', 'bladder',
 'core temperature',
 'bladr',
 'foley ',
 'gastric tube, oral',
 'bldr'
 'foley',  'f.',  'foley',
 'bldr',
 'catheter',
 'pa cath',
 'core central line',
 '.swan'
        ],
    'Rectal':[
 'rc',
 'rctal',
 're',
 'rec',
 'rec ',
 'rec.',
 'recatl',
 'recrtal',
 'rect',
 'recta',
 'recta;',
 'rectaal',
 'rectal',
 'rectal ',
 'rectal  probe',
 'rectal temp probe and foley temp probe',
 'rectal;',
 'rectall',
 'rectally',
 'rectoal',
 'rectsl',
 'r',
 'r.',
 '.rec',
 '.rectal'
    ],
    'Axillary':[
        'axillary',
        'axilla',
        'ax',
         'axllry',
        '.ax',
        'axill',
        'axillary '
        ],
    'Tympanic':[
        'tympanic',
        'tym',
        'tymp'
                ],
}

In [ ]:
%%bigquery eicu_temperature --project physionet-data-403309

SELECT * FROM (
    SELECT
    patientunitstayid AS stay_id,
    nursingchartoffset,
    -- nursingchartentryoffset,
    nursingchartcelltypevallabel,
    nursingchartcelltypevalname,
    nursingchartvalue
    FROM `physionet-data.eicu_crd.nursecharting`
    WHERE nursingchartcelltypevallabel in ('Temperature')
)

PIVOT (
  ANY_VALUE(nursingchartvalue)
  FOR nursingchartcelltypevalname IN (
            'Temperature (C)' as Temp_C,
            'Temperature (F)' as Temp_F,
            'Temperature Location' as Temp_Location
      )
)

WHERE Temp_Location IS NOT NULL

In [ ]:
len(eicu_temperature)

4013465

In [ ]:
with pd.option_context('display.max_rows',None):
  display(eicu_temperature.head())

,icustay_id,nursingchartoffset,nursingchartcelltypevallabel,Temp_C,Temp_F,Temp_Location
0,3211511,1111,Temperature,39.0,102.2,axillary
1,2292631,153,Temperature,35.0,95.0,rectal
2,3126514,1670,Temperature,32.6,90.7,rectal
3,2155321,1425,Temperature,32.9,91.2,core
4,3199153,2302,Temperature,34.4,93.9,axillary


In [ ]:
eicu_temperature['Temp_Location'] = eicu_temperature['Temp_Location'].str.lower()

for col in [
    'stay_id',
    'nursingchartoffset',
    'Temp_C',
    'Temp_F'
    ]:
  eicu_temperature[col] = pd.to_numeric(eicu_temperature[col], downcast='integer')

In [ ]:
with pd.option_context('display.max_rows',20):
  display(eicu_temperature['Temp_Location'].str.lower().value_counts().head(20))

Map temperature locations

In [ ]:
val_glom = []
for key in dict_mappings['eicu']:
  print(key)
  val_glom.extend(dict_mappings['eicu'][key]) #variable with temperture measurment location

Temporal
Oral
Esophageal
Core
Rectal
Axillary
Tympanic


In [ ]:
display(val_glom)

##Process data

In [ ]:
base_date = pd.Timestamp('2014-01-01')
eicu_temperature_cleaned = eicu_temperature.loc[
    eicu_temperature['Temp_Location'].str.lower().isin(val_glom), :].copy()

eicu_temperature_cleaned['datetime_charted'] = base_date + pd.to_timedelta(eicu_temperature_cleaned['nursingchartoffset'], unit='m')

for key in dict_mappings['eicu']:
  key_count = eicu_temperature_cleaned['Temp_Location'].isin(dict_mappings['eicu'][key]).sum()
  print(key, key_count, str(round(key_count/len(eicu_temperature) * 100, 2)) + '%' )

  eicu_temperature_cleaned.loc[
    eicu_temperature_cleaned['Temp_Location'].isin(dict_mappings['eicu'][key]),
      'Temp_Location_mapped' ] = key

In [ ]:
eicu_temperature_cleaned

,stay_id,nursingchartoffset,nursingchartcelltypevallabel,Temp_C,Temp_F,Temp_Location,datetime_charted,Temp_Location_mapped
0,2708819,5579,Temperature,35.00,95.0,oral,2014-01-04 20:59:00,Oral
2,337817,120,Temperature,97.00,206.6,core,2014-01-01 02:00:00,Core
3,3157967,332,Temperature,36.70,98.0,.ta,2014-01-01 05:32:00,Temporal
4,380455,17233,Temperature,97.80,208.0,core,2014-01-12 23:13:00,Core
5,1583138,190,Temperature,32.60,90.7,core,2014-01-01 03:10:00,Core
...,...,...,...,...,...,...,...,...
4013460,1636525,14043,Temperature,38.27,100.9,oral,2014-01-10 18:03:00,Oral
4013461,620388,14310,Temperature,38.30,100.9,core,2014-01-10 22:30:00,Core
4013462,594168,24242,Temperature,38.30,100.9,axillary,2014-01-17 20:02:00,Axillary
4013463,3077220,646,Temperature,38.30,100.9,core,2014-01-01 10:46:00,Core


Make (all) pairs

In [ ]:
eicu_temperature_cleaned = eicu_temperature_cleaned[['stay_id', 'Temp_C', 'nursingchartoffset', 'datetime_charted', 'Temp_Location_mapped']]

In [ ]:
eicu_temp_reference = eicu_temperature_cleaned[(eicu_temperature_cleaned['Temp_Location_mapped'] == 'Oral') |
                                    (eicu_temperature_cleaned['Temp_Location_mapped'] == 'Esophageal') |
                                    (eicu_temperature_cleaned['Temp_Location_mapped'] == 'Rectal') |
                                    (eicu_temperature_cleaned['Temp_Location_mapped'] == 'Core') |
                                    (eicu_temperature_cleaned['Temp_Location_mapped'] == 'Axillary')]
eicu_temp_reference = eicu_temp_reference.rename(columns={
                                                'Temp_C':'temperature_site_1_temperature',
                                                'datetime_charted':'temperature_site_1_charttime',
                                                'Temp_Location_mapped': 'temperature_site_1_temperature_site',
                                                'nursingchartoffset': 'nursingchartoffset_1'
                                                }).sort_values(by=[
                                                'temperature_site_1_charttime'
                                                ])
display(len(eicu_temp_reference))
eicu_temp_reference = eicu_temp_reference.loc[eicu_temp_reference.notnull().all(axis=1),:]
display(len(eicu_temp_reference))

eicu_temp_IR = eicu_temperature_cleaned[(eicu_temperature_cleaned['Temp_Location_mapped'] == 'Tympanic') |
                             (eicu_temperature_cleaned['Temp_Location_mapped'] == 'Temporal')]
eicu_temp_IR = eicu_temp_IR.rename(columns={'Temp_C':'temperature_site_2_temperature',
                                                'datetime_charted':'temperature_site_2_charttime',
                                                'Temp_Location_mapped': 'temperature_site_2_temperature_site',
                                                'nursingchartoffset': 'nursingchartoffset_2'
                                                }).sort_values(by=[
                                                'temperature_site_2_charttime'
                                                ])
display(len(eicu_temp_IR))
eicu_temp_IR = eicu_temp_IR.loc[eicu_temp_IR.notnull().all(axis=1),:]
display(len(eicu_temp_IR))

3269726

3200084

593367

589896

In [ ]:
eicu_pairs = eicu_temp_reference.merge(
                        eicu_temp_IR,
                        how='inner',
                        on='stay_id'
                        )

display(len(eicu_pairs)) #9,905,943

9905943

In [ ]:
eicu_pairs

,stay_id,temperature_site_1_temperature,nursingchartoffset_1,temperature_site_1_charttime,temperature_site_1_temperature_site,temperature_site_2_temperature,nursingchartoffset_2,temperature_site_2_charttime,temperature_site_2_temperature_site
0,1591696,36.9,-85226,2013-11-02 19:34:00,Rectal,36.7,-93525,2013-10-28 01:15:00,Tympanic
1,1591696,36.9,-85226,2013-11-02 19:34:00,Rectal,37.2,-93394,2013-10-28 03:26:00,Tympanic
2,1591696,36.9,-85226,2013-11-02 19:34:00,Rectal,37.3,-93364,2013-10-28 03:56:00,Tympanic
3,1591696,36.9,-85226,2013-11-02 19:34:00,Rectal,35.9,-93184,2013-10-28 06:56:00,Tympanic
4,1591696,36.9,-85226,2013-11-02 19:34:00,Rectal,35.6,-93081,2013-10-28 08:39:00,Tympanic
...,...,...,...,...,...,...,...,...,...
9905938,1581310,36.8,45957,2014-02-01 21:57:00,Oral,36.9,48665,2014-02-03 19:05:00,Temporal
9905939,1581310,36.8,45957,2014-02-01 21:57:00,Oral,36.8,48851,2014-02-03 22:11:00,Temporal
9905940,1581310,36.8,45957,2014-02-01 21:57:00,Oral,36.3,49091,2014-02-04 02:11:00,Temporal
9905941,1581310,36.8,45957,2014-02-01 21:57:00,Oral,36.6,49413,2014-02-04 07:33:00,Temporal


In [ ]:
eicu_pairs['temperature_diff'] = (eicu_pairs['temperature_site_1_temperature'] - eicu_pairs['temperature_site_2_temperature'])
eicu_pairs['time_diff_hours'] = (eicu_pairs['temperature_site_1_charttime'] - eicu_pairs['temperature_site_2_charttime'])
eicu_pairs['time_diff_hours'] = (abs(eicu_pairs['time_diff_hours']).dt.total_seconds() / 3600) #.astype(int)

In [ ]:
with pd.option_context('display.max_columns', None):
    display(eicu_pairs)

,stay_id,temperature_site_1_temperature,temperature_site_1_charttime,temperature_site_1_temperature_site,temperature_site_2_temperature,temperature_site_2_charttime,temperature_site_2_temperature_site,temperature_diff,time_diff_hours
0,1591696,36.9,2013-11-02 19:34:00,Rectal,36.7,2013-10-28 01:15:00,Tympanic,0.2,138.316667
1,1591696,36.9,2013-11-02 19:34:00,Rectal,37.2,2013-10-28 03:26:00,Tympanic,-0.3,136.133333
2,1591696,36.9,2013-11-02 19:34:00,Rectal,37.3,2013-10-28 03:56:00,Tympanic,-0.4,135.633333
3,1591696,36.9,2013-11-02 19:34:00,Rectal,35.9,2013-10-28 06:56:00,Tympanic,1.0,132.633333
4,1591696,36.9,2013-11-02 19:34:00,Rectal,35.6,2013-10-28 08:39:00,Tympanic,1.3,130.916667
...,...,...,...,...,...,...,...,...,...
9905938,1581310,36.8,2014-02-01 21:57:00,Oral,36.9,2014-02-03 19:05:00,Temporal,-0.1,45.133333
9905939,1581310,36.8,2014-02-01 21:57:00,Oral,36.8,2014-02-03 22:11:00,Temporal,0.0,48.233333
9905940,1581310,36.8,2014-02-01 21:57:00,Oral,36.3,2014-02-04 02:11:00,Temporal,0.5,52.233333
9905941,1581310,36.8,2014-02-01 21:57:00,Oral,36.6,2014-02-04 07:33:00,Temporal,0.2,57.600000


In [ ]:
project_id = "physionet-data-403309"
dataset_id = "pairs"

final_tables = {"eicu_original": eicu_pairs}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 6159.04it/s]


Pairs filtered by temperature

In [ ]:
eicu_pairs_temp = eicu_pairs[(eicu_pairs['temperature_site_1_temperature'] > 30.0) &
                            (eicu_pairs['temperature_site_2_temperature'] > 30.0) &
                            (eicu_pairs['temperature_site_1_temperature'] < 45.0) &
                            (eicu_pairs['temperature_site_2_temperature'] < 45.0)]
display(len(eicu_pairs_temp)) #9,892,300

9892300

In [ ]:
project_id = "physionet-data-403309"
dataset_id = "pairs"

final_tables = {"eicu_temp": eicu_pairs_temp}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 6241.52it/s]


Pairs filtered by temperature and window

In [ ]:
eicu_temp_w = eicu_pairs_temp[(eicu_pairs_temp['time_diff_hours'] <= 4)]
display(len(eicu_temp_w)) #98,834

98834

In [ ]:
project_id = "physionet-data-403309"
dataset_id = "pairs"

final_tables = {"eicu_temp_w": eicu_temp_w}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 1529.09it/s]


Pairs filtered by temperature, window and site

In [ ]:
eicu_temp_w_site = eicu_temp_w[((eicu_temp_w['temperature_site_1_temperature_site'] == 'Oral') |
                                    (eicu_temp_w['temperature_site_1_temperature_site'] == 'Esophageal') |
                                    (eicu_temp_w['temperature_site_1_temperature_site'] == 'Rectal')) &
                                    ((eicu_temp_w['temperature_site_2_temperature_site'] == 'Tympanic') |
                                    (eicu_temp_w['temperature_site_2_temperature_site'] == 'Temporal'))]
display(len(eicu_temp_w_site)) #62,242

62242

In [ ]:
eicu_temp_w_site['patientunitstayid'] = eicu_temp_w_site['stay_id']

In [ ]:
project_id = "physionet-data-403309"
dataset_id = "pairs"

final_tables = {"eicu_temp_w_site2": eicu_temp_w_site}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 5915.80it/s]


Merge temperatures and patients' information

In [ ]:
eicu_temp_pat = eicu_temp_w_site.merge(
                  eicu_pat,
                  on=['stay_id'],
                  how='left'
                    )

display(len(eicu_temp_pat)) #62,242

62242

In [ ]:
with pd.option_context('display.max_columns', None):
    display(eicu_temp_pat.head(5))

,stay_id,temperature_site_1_temperature,nursingchartoffset_1,temperature_site_1_charttime,temperature_site_1_temperature_site,temperature_site_2_temperature,nursingchartoffset_2,temperature_site_2_charttime,temperature_site_2_temperature_site,temperature_diff,time_diff_hours,patientunitstayid,subject_id,hospital_admission_id,source_db,hospitalid,numbedscategory,teachingstatus,region,age_admission,sex_female,weight_admission,height_admission,BMI_admission,datetime_hospital_admit,date_hospital_discharge,date_icu_admit,date_icu_discharge,los_hospital,los_ICU,comorbidity_score_name,comorbidity_score_value,in_hospital_mortality,race_ethnicity
0,1591696,36.9,-85226,2013-11-02 19:34:00,Rectal,37.1,-85415,2013-11-02 16:25:00,Tympanic,-0.2,3.150000,1591696,016-24466,1232221,eicu,252,>= 500,True,Midwest,61,0,105.6,190.5,29.098725,-93526,15611,0,10563,75.789583,7.333333,Charlson,6,1,Black
1,1591696,36.9,-85226,2013-11-02 19:34:00,Rectal,36.7,-85092,2013-11-02 21:48:00,Tympanic,0.2,2.233333,1591696,016-24466,1232221,eicu,252,>= 500,True,Midwest,61,0,105.6,190.5,29.098725,-93526,15611,0,10563,75.789583,7.333333,Charlson,6,1,Black
2,1591696,36.4,-63293,2013-11-18 01:07:00,Oral,36.5,-63354,2013-11-18 00:06:00,Tympanic,-0.1,1.016667,1591696,016-24466,1232221,eicu,252,>= 500,True,Midwest,61,0,105.6,190.5,29.098725,-93526,15611,0,10563,75.789583,7.333333,Charlson,6,1,Black
3,1591696,36.9,-63165,2013-11-18 03:15:00,Oral,36.5,-63354,2013-11-18 00:06:00,Tympanic,0.4,3.150000,1591696,016-24466,1232221,eicu,252,>= 500,True,Midwest,61,0,105.6,190.5,29.098725,-93526,15611,0,10563,75.789583,7.333333,Charlson,6,1,Black
4,1591696,36.4,-60511,2013-11-19 23:29:00,Oral,36.3,-60376,2013-11-20 01:44:00,Temporal,0.1,2.250000,1591696,016-24466,1232221,eicu,252,>= 500,True,Midwest,61,0,105.6,190.5,29.098725,-93526,15611,0,10563,75.789583,7.333333,Charlson,6,1,Black


In [ ]:
project_id = "physionet-data-403309"
dataset_id = "pairs_pat"

final_tables = {"eicu_temp_w_site": eicu_temp_pat}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 893.36it/s]


Query with pairs + variables

In [ ]:
temp_time = "nursingchartoffset_1"
pairs_table = "physionet-data-403309.pairs_pat.eicu_temp_w_site"

sheet_id = "1zgW22LTDMR60Al9da7R7UCRdfLGJ3HPoe3J-H-rz9Mw"
gid = "1183670352"
vars_df = get_vars_df(sheet_id, gid)

query = create_complete_query(pairs_table, vars_df, temp_time)
client = bigquery.Client(project='physionet-data-403309')
eicu_df = client.query(query).to_dataframe()

In [ ]:
with pd.option_context('display.max_columns', None):
    display(eicu_df)

,stay_id,temperature_site_1_temperature,nursingchartoffset_1,temperature_site_1_charttime,temperature_site_1_temperature_site,temperature_site_2_temperature,nursingchartoffset_2,temperature_site_2_charttime,temperature_site_2_temperature_site,temperature_diff,time_diff_hours,patientunitstayid,subject_id,hospital_admission_id,source_db,hospitalid,numbedscategory,teachingstatus,region,age_admission,sex_female,weight_admission,height_admission,BMI_admission,datetime_hospital_admit,date_hospital_discharge,date_icu_admit,date_icu_discharge,los_hospital,los_ICU,comorbidity_score_name,comorbidity_score_value,in_hospital_mortality,race_ethnicity,delta_vitals_heart_rate,vitals_heart_rate,delta_vitals_resp_rate,vitals_resp_rate,delta_vitals_mbp_ni,vitals_mbp_ni,delta_vitals_sbp_ni,vitals_sbp_ni,delta_vitals_dbp_ni,vitals_dbp_ni,delta_vitals_mbp_i,vitals_mbp_i,delta_vitals_sbp_i,vitals_sbp_i,delta_vitals_dbp_i,vitals_dbp_i,delta_vitals_spo2,vitals_spo2,delta_cbc_hemoglobin,cbc_hemoglobin,delta_cbc_hematocrit,cbc_hematocrit,delta_cbc_mch,cbc_mch,delta_cbc_mchc,cbc_mchc,delta_cbc_mcv,cbc_mcv,delta_cbc_platelet,cbc_platelet,delta_cbc_rbc,cbc_rbc,delta_cbc_rdw,cbc_rdw,delta_cbc_wbc,cbc_wbc,delta_coag_fibrinogen,coag_fibrinogen,delta_coag_inr,coag_inr,delta_coag_pt,coag_pt,delta_coag_ptt,coag_ptt,delta_bmp_sodium,bmp_sodium,delta_bmp_potassium,bmp_potassium,delta_bmp_chloride,bmp_chloride,delta_bmp_bicarbonate,bmp_bicarbonate,delta_bmp_bun,bmp_bun,delta_bmp_creatinine,bmp_creatinine,delta_bmp_glucose,bmp_glucose,delta_bmp_aniongap,bmp_aniongap,delta_bmp_calcium,bmp_calcium,delta_bmp_lactate,bmp_lactate,delta_hfp_alt,hfp_alt,delta_hfp_alp,hfp_alp,delta_hfp_ast,hfp_ast,delta_hfp_bilirubin_total,hfp_bilirubin_total,delta_hfp_bilirubin_direct,hfp_bilirubin_direct,delta_hfp_albumin,hfp_albumin,delta_others_ck_cpk,others_ck_cpk,delta_others_ck_mb,others_ck_mb,delta_others_ld_ldh,others_ld_ldh,delta_sofa_past_overall_24hr,sofa_past_overall_24hr,delta_sofa_past_coagulation_24hr,sofa_past_coagulation_24hr,delta_sofa_past_liver_24hr,sofa_past_liver_24hr,delta_sofa_past_cardiovascular_24hr,sofa_past_cardiovascular_24hr,delta_sofa_past_cns_24hr,sofa_past_cns_24hr,delta_sofa_past_renal_24hr,sofa_past_renal_24hr,delta_sofa_future_overall_24hr,sofa_future_overall_24hr,delta_sofa_future_coagulation_24hr,sofa_future_coagulation_24hr,delta_sofa_future_liver_24hr,sofa_future_liver_24hr,delta_sofa_future_cardiovascular_24hr,sofa_future_cardiovascular_24hr,delta_sofa_future_cns_24hr,sofa_future_cns_24hr,delta_sofa_future_renal_24hr,sofa_future_renal_24hr
0,201262,37.7,-176,2013-12-31 21:04:00,Oral,36.8,-31,2013-12-31 23:29:00,Temporal,0.9000000000000057,2.4166666666666665,201262,002-48805,175595,eicu,56,<100,False,Midwest,90,1,76.4,137.2,40.58683031729978,-18,6007,0,3096,4.184027777777778,2.1666666666666665,Charlson,6,0,White,0,112.0,0,20.0,<NA>,NaN,0,98.0,0,52.0,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1556,3,1556,1,1556,0,1556,1,1556,0,1556,1
1,172234,37.3,-2183,2013-12-30 11:37:00,Oral,36.3,-2096,2013-12-30 13:04:00,Temporal,1.0,1.45,172234,002-21687,152927,eicu,58,100 - 249,False,Midwest,68,0,90.9,185.0,26.55953250547845,-2202,4282,0,1203,4.502777777777777,0.8333333333333334,Charlson,3,0,White,0,74.0,0,20.0,<NA>,NaN,0,119.0,0,60.0,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,<NA>,-98,9.9,-98,30.4,-98,28.6,-98,32.6,-98,87.9,-98,251.0,-98,3.46,-98,15.2,-98,15.5,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,-98,133.0,-98,3.6,-98,98.0,-98,30.0,-98,20.0,-98,1.10,-98,128.0,-98,9.0,-98,8.2,<NA>,NaN,-98,16.0,-98,134.0,-98,14.0,-98,1.1,<NA>,NaN,-98,2.8,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,

In [ ]:
project_id = "physionet-data-403309"
dataset_id = "h_merged"

final_tables_merged = {"eicu_df":  eicu_df}

for t_name, t_df in final_tables_merged.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 8507.72it/s]


#Merge vertically

In [ ]:
%%bigquery mimic_iv_all --project=physionet-data-403309

SELECT *
FROM `physionet-data-403309.h_merged.mimic_iv_temp_w_site`

In [ ]:
%%bigquery mimic_iii_all --project=physionet-data-403309

SELECT *
FROM `physionet-data-403309.h_merged.mimic_iii_temp_w_site`

In [ ]:
%%bigquery eicu_all --project=physionet-data-403309

SELECT *
FROM `physionet-data-403309.h_merged.eicu_df`

In [ ]:
print(len(mimic_iii_all), len(mimic_iv_all), len(eicu_all))

114 21277 62242


In [ ]:
base_date = pd.Timestamp('2014-01-01')
eicu_all = eicu_all.rename(columns={
                            'date_hospital_discharge': 'datetime_hospital_discharge',
                            'date_icu_admit': 'datetime_icu_admit',
                            'date_icu_discharge': 'datetime_icu_discharge',
                            })

eicu_all['datetime_hospital_admit'] = base_date + pd.to_timedelta(eicu_all['datetime_hospital_admit'].astype(int), unit='m')
eicu_all['datetime_hospital_discharge'] = base_date + pd.to_timedelta(eicu_all['datetime_hospital_discharge'].astype(int), unit='m')
eicu_all['datetime_icu_admit'] = base_date + pd.to_timedelta(eicu_all['datetime_icu_admit'].astype(int), unit='m')
eicu_all['datetime_icu_discharge'] = base_date + pd.to_timedelta(eicu_all['datetime_icu_discharge'].astype(int), unit='m')

In [ ]:
columns_names = [
    'subject_id', 'hospital_admission_id', 'icustay_id',
    'source_db', 'hospitalid', 'numbedscategory', 'teachingstatus', 'region',
    'age_admission', 'sex_female', 'weight_admission', 'height_admission',
    'BMI_admission', 'datetime_hospital_admit', 'datetime_hospital_discharge', 'datetime_icu_admit',
    'datetime_icu_discharge', 'los_hospital', 'los_ICU', 'comorbidity_score_name', 'comorbidity_score_value',
    'in_hospital_mortality', 'race_ethnicity',
    'temperature_site_1_charttime','temperature_site_1_temperature','temperature_site_1_temperature_site',
    'temperature_site_2_charttime','temperature_site_2_temperature','temperature_site_2_temperature_site',
    'temperature_diff','time_diff_hours',
    'vitals_heart_rate', 'vitals_resp_rate',
    'vitals_mbp_ni', 'vitals_sbp_ni', 'vitals_dbp_ni', 'vitals_mbp_i', 'vitals_sbp_i', 'vitals_dbp_i',
    'delta_vitals_spo2', 'vitals_spo2',
    'cbc_hemoglobin', 'cbc_hematocrit', 'cbc_mch', 'cbc_mchc', 'cbc_mcv', 'cbc_platelet',
    'cbc_rbc', 'cbc_rdw', 'cbc_wbc', 'coag_fibrinogen', 'coag_inr', 'coag_pt', 'coag_ptt', 'bmp_sodium',
    'bmp_potassium', 'bmp_chloride', 'bmp_bicarbonate', 'bmp_bun', 'bmp_creatinine', 'bmp_glucose',
    'bmp_aniongap', 'bmp_calcium', 'bmp_lactate', 'hfp_alt', 'hfp_alp', 'hfp_ast', 'hfp_bilirubin_total',
    'hfp_bilirubin_direct', 'hfp_albumin', 'others_ck_cpk', 'others_ck_mb', 'others_ld_ldh',
    'sofa_past_overall_24hr', 'sofa_past_coagulation_24hr',
    'sofa_past_liver_24hr', 'sofa_past_cardiovascular_24hr', 'sofa_past_cns_24hr', 'sofa_past_renal_24hr',
    'sofa_future_overall_24hr', 'sofa_future_coagulation_24hr', 'sofa_future_liver_24hr',
    'sofa_future_cardiovascular_24hr', 'sofa_future_cns_24hr', 'sofa_future_renal_24hr'
]

In [ ]:
col_map_mimic_iii = dict(zip(['admission_age'], ['age_admission']))
col_map_mimic_iv = dict(zip(['stay_id'], ['icustay_id']))
col_map_mimic_eicu = dict(zip(['stay_id','date_hospital_discharge', 'date_icu_admit','date_icu_discharge'], \
                            ['icustay_id', 'datetime_hospital_discharge', 'datetime_icu_admit','datetime_icu_discharge']))

In [ ]:
mimic_iii_all.rename(columns=col_map_mimic_iii, inplace=True)
mimic_iv_all.rename(columns=col_map_mimic_iv, inplace=True)
eicu_all.rename(columns=col_map_mimic_eicu, inplace=True)

In [ ]:
mimic_iii_all = mimic_iii_all[[column for column in columns_names]]
mimic_iv_all = mimic_iv_all[[column for column in columns_names]]
eicu_all = eicu_all[[column for column in columns_names]]

In [ ]:
print((mimic_iv_all.columns == mimic_iii_all.columns).all())
print((mimic_iv_all.columns == eicu_all.columns).all())

True
True


In [ ]:
vertical_merged_df = pd.concat([mimic_iii_all, mimic_iv_all, eicu_all], ignore_index=True)

In [ ]:
len(vertical_merged_df)

83633

In [ ]:
# harmonize missingness
vertical_merged_df = vertical_merged_df.replace("___", np.nan)

# Replace instances of None with NaN
vertical_merged_df = vertical_merged_df.replace("None", np.nan)

data_types = {
    'subject_id': str,
    'hospital_admission_id': int,
    'age_admission': int,
    'icustay_id': int,
    'temperature_site_1_temperature': float,
    'temperature_site_2_temperature': float,
    'datetime_hospital_admit': str,
    'datetime_hospital_discharge': str,
    'datetime_icu_admit': str,
    'datetime_icu_discharge': str
}

vertical_merged_df = vertical_merged_df.astype(data_types)

# Display the merged DataFrame
display(vertical_merged_df.head(2))

,subject_id,hospital_admission_id,icustay_id,source_db,hospitalid,numbedscategory,teachingstatus,region,age_admission,sex_female,...,sofa_past_liver_24hr,sofa_past_cardiovascular_24hr,sofa_past_cns_24hr,sofa_past_renal_24hr,sofa_future_overall_24hr,sofa_future_coagulation_24hr,sofa_future_liver_24hr,sofa_future_cardiovascular_24hr,sofa_future_cns_24hr,sofa_future_renal_24hr
0,91591,107671,210497,mimic_iii,9999,>= 500,True,Northeast,21,1,...,0,0,0,0,7,0,0,1,3,3
1,91591,107671,210497,mimic_iii,9999,>= 500,True,Northeast,21,1,...,0,0,2,3,4,0,0,1,3,0


In [ ]:
vertical_merged_df = vertical_merged_df.sort_values(by=['source_db', 'subject_id', 'hospital_admission_id', 'icustay_id'])

vertical_merged_df['unique_subject_id'] = pd.factorize(vertical_merged_df['source_db'] + vertical_merged_df['subject_id'].astype(str))[0] % 1000000
vertical_merged_df['unique_hospital_admission_id'] = pd.factorize(vertical_merged_df['source_db'] + vertical_merged_df['hospital_admission_id'].astype(str))[0] % 1000000
vertical_merged_df['unique_icustay_id'] = pd.factorize(vertical_merged_df['source_db'] + vertical_merged_df['icustay_id'].astype(str))[0] % 1000000

# move the unique columns to the front
final_cols = ['unique_subject_id', 'unique_hospital_admission_id', 'unique_icustay_id'] + [col for col in vertical_merged_df.columns if col not in ['unique_subject_id', 'unique_hospital_admission_id', 'unique_icustay_id']]
final_df = vertical_merged_df[final_cols]

In [ ]:
# Plot Missingness Bar Chart
mdf_sorted = msno.nullity_sort(final_df, sort='descending') # or sort='ascending'
bar = msno.bar(mdf_sorted, filter='top', sort='descending', fontsize = 18,  label_rotation=45)

# Modify y-axis labels to percentages
plt.yticks(np.arange(0, 1.1, 0.1), [str(int(x * 100)) + '%' for x in np.arange(0, 1.1, 0.1)])
plt.title("Covariates' Completeness\n", fontsize=30)
plt.show()

In [ ]:
with pd.option_context('display.max_columns', None):
    display(final_df.reset_index())

,index,unique_subject_id,unique_hospital_admission_id,unique_icustay_id,subject_id,hospital_admission_id,icustay_id,source_db,hospitalid,numbedscategory,teachingstatus,region,age_admission,sex_female,weight_admission,height_admission,BMI_admission,datetime_hospital_admit,datetime_hospital_discharge,datetime_icu_admit,datetime_icu_discharge,los_hospital,los_ICU,comorbidity_score_name,comorbidity_score_value,in_hospital_mortality,race_ethnicity,temperature_site_1_charttime,temperature_site_1_temperature,temperature_site_1_temperature_site,temperature_site_2_charttime,temperature_site_2_temperature,temperature_site_2_temperature_site,temperature_diff,time_diff_hours,vitals_heart_rate,vitals_resp_rate,vitals_mbp_ni,vitals_sbp_ni,vitals_dbp_ni,vitals_mbp_i,vitals_sbp_i,vitals_dbp_i,delta_vitals_spo2,vitals_spo2,cbc_hemoglobin,cbc_hematocrit,cbc_mch,cbc_mchc,cbc_mcv,cbc_platelet,cbc_rbc,cbc_rdw,cbc_wbc,coag_fibrinogen,coag_inr,coag_pt,coag_ptt,bmp_sodium,bmp_potassium,bmp_chloride,bmp_bicarbonate,bmp_bun,bmp_creatinine,bmp_glucose,bmp_aniongap,bmp_calcium,bmp_lactate,hfp_alt,hfp_alp,hfp_ast,hfp_bilirubin_total,hfp_bilirubin_direct,hfp_albumin,others_ck_cpk,others_ck_mb,others_ld_ldh,sofa_past_overall_24hr,sofa_past_coagulation_24hr,sofa_past_liver_24hr,sofa_past_cardiovascular_24hr,sofa_past_cns_24hr,sofa_past_renal_24hr,sofa_future_overall_24hr,sofa_future_coagulation_24hr,sofa_future_liver_24hr,sofa_future_cardiovascular_24hr,sofa_future_cns_24hr,sofa_future_renal_24hr
0,30291,0,0,0,002-10009,193705,224606,eicu,71,100 - 249,False,Midwest,76,1,nan,160.0,nan,2013-12-31 21:01:00,2014-01-06 16:23:00,2014-01-01 00:00:00,2014-01-03 21:46:00,5.8069444444444445,2.9166666666666665,Charlson,7,0,White,2014-01-01 02:17:00,37.30,Oral,2013-12-31 22:42:00,37.10,Temporal,0.19999999999999574,3.5833333333333335,117.0,29.0,nan,94.0,54.0,68.0,nan,nan,-4,88,8.6,27.5,29.1,31.3,92.9,32.0,2.96,19.7,19.2,nan,1.5,16.1,38.0,143.0,3.4,113.0,18.0,25.0,1.0,102.0,15.0,6.1,1.8,14.0,102.0,32.0,1.1,nan,2.2,nan,nan,nan,4,3,0,1,0,0,6,2,0,1,0,0
1,30292,0,0,0,002-10009,193705,224606,eicu,71,100 - 249,False,Midwest,76,1,nan,160.0,nan,2013-12-31 21:01:00,2014-01-06 16:23:00,2014-01-01 00:00:00,2014-01-03 21:46:00,5.8069444444444445,2.9166666666666665,Charlson,7,0,White,2014-01-01 02:17:00,37.30,Oral,2013-12-31 22:53:00,36.90,Temporal,0.3999999999999986,3.4,117.0,29.0,nan,94.0,54.0,68.0,nan,nan,-4,88,8.6,27.5,29.1,31.3,92.9,32.0,2.96,19.7,19.2,nan,1.5,16.1,38.0,143.0,3.4,113.0,18.0,25.0,1.0,102.0,15.0,6.1,1.8,14.0,102.0,32.0,1.1,nan,2.2,nan,nan,nan,4,3,0,1,0,0,6,2,0,1,0,0
2,61921,0,0,0,002-10009,193705,224606,eicu,71,100 - 249,False,Midwest,76,1,nan,160.0,nan,2013-12-31 21:01:00,2014-01-06 16:23:00,2014-01-01 00:00:00,2014-01-03 21:46:00,5.8069444444444445,2.9166666666666665,Charlson,7,0,White,2013-12-31 23:57:00,36.80,Oral,2013-12-31 22:42:00,37.10,Temporal,-0.30000000000000426,1.25,112.0,26.0,nan,91.0,50.0,67.0,nan,nan,<NA>,<NA>,12.2,39.0,29.2,31.3,93.3,52.0,4.18,20.5,35.2,nan,nan,nan,nan,136.0,4.4,97.0,21.0,24.0,1.1,82.0,22.0,9.1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,6,2,0,1,0,0
3,61922,0,0,0,002-10009,193705,224606,eicu,71,100 - 249,False,Midwest,76,1,nan,160.0,nan,2013-12-31 21:01:00,2014-01-06 16:23:00,2014-01-01 00:00:00,2014-01-03 21:46:00,5.8069444444444445,2.9166666666666665,Charlson,7,0,White,2013-12-31 23:57:00,36.80,Oral,2013-12-31 22:53:00,36.90,Temporal,-0.10000000000000142,1.0666666666666667,112.0,26.0,nan,91.0,50.0,67.0,nan,nan,<NA>,<NA>,12.2,39.0,29.2,31.3,93.3,52.0,4.18,20.5,35.2,nan,nan,nan,nan,136.0,4.4,97.0,21.0,24.0,1.1,82.0,22.0,9.1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,6,2,0,1,0,0
4,31942,1,1,1,002-10034,141169,157016,eicu,63,100 - 249,False,Midwest,23,1,63.5,162.6,24.017767852955284,2013-12-29 16:29:00,2014-01-03 21:32:00,2014-01-01 00:00:00,2014-01-03 21:32:00,5.210416666666666,2.9166666666666665,Charlson,0,0,White,2014-01-01 03:53:00,37.20,Oral,2014-01-01 00:02:00,36.40,Temporal,0

final

In [ ]:
project_id = "physionet-data-403309"
dataset_id = "h_merged"

final_tables = {"vmerged_all": final_df}

for t_name, t_df in final_tables.items():

  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 7384.34it/s]


In [ ]:
folder_path = '/content/drive/My Drive/'

In [ ]:
final_df.to_csv(folder_path+'vmerged_all.csv', index=False)
files.download(folder_path+'vmerged_all.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>